# Strategy Optimization Notebook
Optimize trading strategy parameters using historical data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import ccxt

# Strategy parameters to optimize
params_grid = {
    'fast_ma': [5, 10, 12, 15],
    'slow_ma': [20, 26, 30, 35],
    'rsi_period': [10, 14, 20],
    'rsi_oversold': [20, 25, 30],
    'rsi_overbought': [70, 75, 80]
}

print('Parameter optimization initialized')

In [ ]:
def optimize_parameters(df, params_grid):
    """Grid search optimization of parameters"""
    best_score = -np.inf
    best_params = {}
    results = []
    
    for fast_ma in params_grid['fast_ma']:
        for slow_ma in params_grid['slow_ma']:
            if fast_ma >= slow_ma:
                continue
            for rsi_period in params_grid['rsi_period']:
                # Calculate indicators
                df['fast_sma'] = df['close'].rolling(fast_ma).mean()
                df['slow_sma'] = df['close'].rolling(slow_ma).mean()
                
                # Generate signals
                df['signal'] = 0
                df.loc[df['fast_sma'] > df['slow_sma'], 'signal'] = 1
                df.loc[df['fast_sma'] < df['slow_sma'], 'signal'] = -1
                
                # Calculate returns
                df['returns'] = df['close'].pct_change()
                df['strategy_returns'] = df['signal'].shift(1) * df['returns']
                
                # Calculate performance
                total_return = df['strategy_returns'].sum() * 100
                sharpe_ratio = df['strategy_returns'].mean() / df['strategy_returns'].std() * np.sqrt(252)
                max_dd = calculate_max_drawdown(df['strategy_returns'])
                win_rate = (df['strategy_returns'] > 0).sum() / len(df['strategy_returns']) * 100
                
                # Scoring function
                score = sharpe_ratio * 0.5 + win_rate * 0.3 + (100 - max_dd) * 0.2
                
                results.append({
                    'fast_ma': fast_ma,
                    'slow_ma': slow_ma,
                    'rsi_period': rsi_period,
                    'total_return': total_return,
                    'sharpe_ratio': sharpe_ratio,
                    'win_rate': win_rate,
                    'max_drawdown': max_dd,
                    'score': score
                })
                
                if score > best_score:
                    best_score = score
                    best_params = {
                        'fast_ma': fast_ma,
                        'slow_ma': slow_ma,
                        'rsi_period': rsi_period
                    }
    
    results_df = pd.DataFrame(results)
    return best_params, best_score, results_df

def calculate_max_drawdown(returns):
    cumsum = (1 + returns).cumprod()
    running_max = cumsum.expanding().max()
    drawdown = (cumsum - running_max) / running_max * 100
    return drawdown.min()

print('Optimization functions defined')

In [ ]:
# Example: Load BTC/USDT historical data and optimize
# best_params, best_score, results = optimize_parameters(df, params_grid)
# print(f'Best Parameters: {best_params}')
# print(f'Best Score: {best_score:.4f}')
# print(results.nlargest(10, 'score'))

## Optimization Results
Top 10 parameter combinations by score

In [ ]:
# Visualization of optimization results
# fig, axes = plt.subplots(2, 2, figsize=(14, 10))
# axes[0, 0].scatter(results['sharpe_ratio'], results['total_return'])
# axes[0, 0].set_xlabel('Sharpe Ratio')
# axes[0, 0].set_ylabel('Total Return (%)')
# axes[0, 0].set_title('Return vs Sharpe Ratio')
# axes[0, 1].scatter(results['max_drawdown'], results['win_rate'])
# axes[0, 1].set_xlabel('Max Drawdown (%)')
# axes[0, 1].set_ylabel('Win Rate (%)')
# axes[1, 0].plot(results.nlargest(20, 'score')['score'].values)
# axes[1, 0].set_title('Top 20 Scores')
# axes[1, 1].bar(range(10), results.nlargest(10, 'score')['win_rate'].values)
# axes[1, 1].set_title('Win Rate - Top 10 Parameters')
# plt.tight_layout()
# plt.show()